#### Problem Description:
* A relatively young bank is growing rapidly in terms of overall customer acquisition.Majority of these are Liability customers with varying sizes of relationship with the bank.The customer base of Asset customers is quite small, and the bank WANTS to grow this base rapidly to bring in more loan business. 

* Specifically, it wants to explore ways of converting its liability customers to Personal Loan customers.

* A campaign the bank ran for liability customers last year showed a healthy conversion rate of over 9% successes. This has encouraged the Retail Marketing department to devise smarter campaigns with better target marketing.

#### Anlaytics Objectives :
	
	
* 1	While designing a new campaign, can we model the previous campaign's customer behavior to 
	analyze what combination of parameters make a customer more likely to 
	accept a personal loan?
	
* 2	There are several special products / facilities the bank offers like CD and security accounts, 
	online services, credit cards, etc. Can we spot any association among these
	for finding cross-selling opportunities?

#### Data Set Description :
* ID:	Customer ID			
* Age:	Customer's age in completed years			
* Experience:	#years of professional experience			
* Income:	Annual income of the customer in thousands of Dollars			
* ZIPCode:	Home Address ZIP code.			Do not use ZIP code
* Family:	Family size of the customer			
* CCAvg:	Avg. spending on credit cards per month in thousands of Dollars		
* Education:	Education Level. 1: Undergrad; 2: Graduate; 3: Advanced/Professional			
* Mortgage:	Value of house mortgage if any. (thousands of Dollars)			
* **Personal Loan:	Did this customer accept the personal loan offered in the last campaign?**			
* Securities Account:	Does the customer have a securities account with the bank?			
* CD Account:	Does the customer have a certificate of deposit (CD) account with the bank?			
* Online:	Does the customer use internet banking facilities?			
* CreditCard:	Does the customer use a credit card issued by UniversalBank?			

#### Note:
* While reading the data set  replace the '?',',' as NAs

#### Experiment :
* Building a Random Forest to predict whether a person takes personal loan or not



In [ ]:
import os
import pandas as pd
path = os.getcwd()
os.chdir(path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

####  Check the dimensions and type

In [ ]:
bank=pd.read_csv("UniversalBank.csv",na_values=["?",","])
print("The number of Rows in the bank data set  ="+str(bank.shape[0]))
print("The number of Columns in the bank data set =" +str(bank.shape[1]))

#### Print Columns names and check the datatypes of columns(dtypes)

In [ ]:
print("The columns in the data set are : \n",list(bank.columns))

In [ ]:
print("The data types of the columns are :\n ",bank.dtypes)

#### Check the top 10 rows to glance the data set 

In [ ]:
bank.head(10)

In [ ]:
bank.tail(10)

#### Check the summary of dataframe(describe())

In [ ]:
bank.describe(include='all')

#### Check the unique levels in the target attribute Personal and also check for the percentage distribution

In [ ]:
bank["Personal Loan"].value_counts()/bank.shape[0]

#### Check the number of unique ZIP Codes present in the dataset 

In [ ]:
print("The number of Unique ZIP Codes in the bank data set",bank['ZIP Code'].value_counts().count())


#### Check the Unique counts of  Family members  present in the dataset

In [ ]:
print("The number of Unique Family members in the bank data set: \n",bank['Family'].value_counts())

#### Check the Unique counts of Education levels present in the dataset

In [ ]:
print("The number of Unique Education levels in the bank data set: \n",bank['Education'].value_counts())

#### Do Necessary changes for the data types from the previous observations

In [ ]:
bank['Education']=bank['Education'].astype('category')
bank['CD Account']=bank['CD Account'].astype('category')
bank['Online']=bank['Online'].astype('category')
bank['CreditCard']=bank['CreditCard'].astype('category')
bank['Securities Account']=bank['Securities Account'].astype('category')
bank['Family']=bank['Family'].astype('category')
bank['ZIP Code']=bank['ZIP Code'].astype('category')
######################################################################################################################
#Use the following code when you have more aolumns
# for column in ['Education', 'CD Account', 'Online']:
#     bank[column]=bank[column].astype('category')
#bank[bank.select_dtypes(['object']).columns] = bank.select_dtypes(['object']).apply(lambda x: x.astype('category')) 
#bank[bank.select_dtypes(['object']).columns] = bank.select_dtypes(['object']).apply(lambda x: x.astype('category')) 
######################################################################################################################

In [ ]:
bank.dtypes

#### Remove the unncessary Columns

In [ ]:
bank=bank.drop(["ID","ZIP Code"],axis=1)

In [ ]:
bank.head(3)

####  Check the missing values 

In [ ]:
bank.isnull().sum()

#### SPLIT THE data in to train and test(use sklearn package)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y=bank["Personal Loan"]
X=bank.drop('Personal Loan', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)  

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

#### Split the numerical and Categorical Attributes

In [ ]:
num_attr = X.select_dtypes(include=['float64','int64']).columns
num_attr

In [ ]:
cat_attr = X.select_dtypes(exclude=['float64','int64']).columns
cat_attr

#### Standardize the data (numerical attributes only)( import StandardScaler) 


In [ ]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
numeric_transformer = Pipeline(memory ='./' ,steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(memory = './',steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

# Now It is time for Model BUilding 

#### Let us see the details of the Random Forest in Sklearn 
* class sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)



* A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True 

#### Parameters
* **n_estimators** : integer, optional (default=10)
    **The number of trees in the forest**.The default value of n_estimators will change from 10 in version 0.20 to 100 in version 0.22.

* **criterion** : string, optional (default=”gini”)
    The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Note: this parameter is tree-specific.

* **max_depth** : integer or None, optional (default=None)
    The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

* **min_samples_split** : int, float, optional (default=2)
    The minimum number of samples required to split an internal node:

    If int, then consider min_samples_split as the minimum number.
    If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.


* **min_samples_leaf** : int, float, optional (default=1)
    The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

    If int, then consider min_samples_leaf as the minimum number.
    If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.


* **min_weight_fraction_leaf** : float, optional (default=0.)
    The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. Samples have equal weight when sample_weight is not provided.

* **max_features** : int, float, string or None, optional (default=”auto”)
    The number of features to consider when looking for the best split:

    If int, then consider max_features features at each split.
    If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split.
    If “auto”, then max_features=sqrt(n_features).
    If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).
    If “log2”, then max_features=log2(n_features).
    If None, then max_features=n_features.
    Note: the search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than max_features features.

* **max_leaf_nodes** : int or None, optional (default=None)
    Grow trees with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.

* **min_impurity_decrease** : float, optional (default=0.)
    A node will be split if this split induces a decrease of the impurity greater than or equal to this value.

  The weighted impurity decrease equation is the following:

    N_t / N * (impurity - N_t_R / N_t * right_impurity
                    - N_t_L / N_t * left_impurity)
    where N is the total number of samples, N_t is the number of samples at the current node, N_t_L is the number of samples in the left child, and N_t_R is the number of samples in the right child.

    N, N_t, N_t_R and N_t_L all refer to the weighted sum, if sample_weight is passed.

* **bootstrap** : boolean, optional (default=True)
    Whether bootstrap samples are used when building trees.

* **oob_score** : bool (default=False)
    Whether to use out-of-bag samples to estimate the generalization accuracy.

* **n_jobs** : int or None, optional (default=None)
    The number of jobs to run in parallel for both fit and predict. None means 1 unless in a joblib.parallel_backend context. -1 means using all processors. See Glossary for more details.

* **random_state** : int, RandomState instance or None, optional (default=None)
    If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

* **verbose** : int, optional (default=0)
    Controls the verbosity when fitting and predicting.

* **warm_start** : bool, optional (default=False)
    When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest. See the Glossary.

* **class_weight** : dict, list of dicts, “balanced”, “balanced_subsample” or None, optional (default=None)
    Weights associated with classes in the form {class_label: weight}. If not given, all classes are supposed to have weight one. For multi-output problems, a list of dicts can be provided in the same order as the columns of y.

    Note that for multioutput (including multilabel) weights should be defined for each class of every column in its own dict. For example, for four-class multilabel classification weights should be [{0: 1, 1: 1}, {0: 1, 1: 5}, {0: 1, 1: 1}, {0: 1, 1: 1}] instead of [{1:1}, {2:5}, {3:1}, {4:1}].

    The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))

    The “balanced_subsample” mode is the same as “balanced” except that weights are computed based on the bootstrap sample for every tree grown.

    For multi-output, the weights of each column of y will be multiplied.

    Note that these weights will be multiplied with sample_weight (passed through the fit method) if sample_weight is specified.

####  Build Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10,max_depth=8)

* Append classifier to preprocessing pipeline.Now we have a full prediction pipeline.

In [ ]:
clf = Pipeline(memory = './',steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score


y_pred = clf.predict(X_train)
print("Train Accuracy = ",accuracy_score(y_train,y_pred))
print("Recall in train = ",recall_score(y_train,y_pred,pos_label=1))

y_pred = clf.predict(X_test)
print("Test Accuracy = ",accuracy_score(y_test,y_pred))
print("Recall on Test = ",recall_score(y_test,y_pred,pos_label=1))

In [ ]:
clf.named_steps['classifier'].feature_importances_

In [ ]:
!ls -la ./joblib/sklearn/pipeline/_fit_transform_one/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
features =clf.steps[1][1].columns
print(features)
importances = clf.named_steps['classifier'].feature_importances_
indices = np.argsort(importances)
print(indices)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()
print([features[i] for i in indices])

#### GridSearch Cross validation

In [ ]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(n_jobs=-1, max_features='sqrt') 
 
# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [9, 18, 27, 36, 45, 54, 63],
           "max_depth" : [2,3,5,7],
           "min_samples_leaf" : [2, 4]}
 
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10)
CV_rfc.fit(X=X_train, y=y_train)
print (CV_rfc.best_score_, CV_rfc.best_params_) 

In [ ]:
y_pred_test=CV_rfc.predict(X_test)
print(accuracy_score(y_test,y_pred_test))

In [ ]:
from sklearn.metrics import recall_score
print(recall_score(y_test,y_pred_test, pos_label=1, average='binary'))

#### Bagging

In [ ]:
clf = BaggingClassifier(n_estimators=10)
clf.fit(X=X_train, y=y_train)

#### Accuracy on testdata

In [ ]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

#### Accuracy on train data

In [ ]:
y_pred = clf.predict(X_train)
print(accuracy_score(y_train,y_pred))

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
                                     n_estimators = 100, max_features = 0.5),
                   param_grid,scoring='recall')
clf.fit(X_train, y_train)

In [ ]:
print(clf.best_score_,clf.best_params_)

In [ ]:

from sklearn.metrics import recall_score
trainpreds=clf.predict(X_train)
print(recall_score(y_train,trainpreds,pos_label=1))# recall
print(accuracy_score(y_train,trainpreds))# accuracy